<a href="https://colab.research.google.com/github/lvllvl/SLAM/blob/main/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Section

## Run script: Connect to google drive

In [1]:
# Mount your drive, to save parts of your model, e.g., checkpoints, etc.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Run Script: Git clone repos

- comma10k repo
- project repo (SLAM)

## clone repo, org folder structures

In [2]:
import os
import shutil

def clone_repo(repo_url, dest_path):
    if os.path.exists(dest_path):
        print(f"Directory {dest_path} already exists. Removing and re-cloning.")
        shutil.rmtree(dest_path)
    os.system(f"git clone {repo_url} {dest_path}")

# Clone repos
clone_repo('https://github.com/lvllvl/SLAM.git', 'SLAM')
clone_repo('https://github.com/commaai/comma10k.git', 'comma10k')


# import os
# import shutil

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def move_files(src, dst):
    if not os.path.exists(src):
        print(f"Source directory {src} does not exist.")
        return
    for filename in os.listdir(src):
        src_file = os.path.join(src, filename)
        dst_file = os.path.join(dst, filename)
        try:
            shutil.move(src_file, dst_file)
        except Exception as e:
            print(f"Error moving {src_file} to {dst_file}: {e}")


def move_selected_files(src, dst, prefix=None, suffix=None):
    """
    Move files from src to dst based on prefix and/or suffix criteria.

    :param src: Source directory
    :param dst: Destination directory
    :param prefix: Prefix to filter files (optional)
    :param suffix: Suffix to filter files (optional)
    """
    if not os.path.exists(src):
        print(f"Source directory {src} does not exist.")
        return
    for filename in os.listdir(src):
        if prefix and not filename.startswith(prefix):
            continue
        if suffix and not filename.endswith(suffix):
            continue
        src_file = os.path.join(src, filename)
        dst_file = os.path.join(dst, filename)
        try:
            shutil.move(src_file, dst_file)
        except Exception as e:
            print(f"Error moving {src_file} to {dst_file}: {e}")


# Create directories for training and validation
create_directory('dataset_root/train/images')
create_directory('dataset_root/train/masks')
create_directory('dataset_root/val/images')
create_directory('dataset_root/val/masks')

# Move images and masks to training folder
move_files('comma10k/imgs', 'dataset_root/train/images')
move_files('comma10k/masks', 'dataset_root/train/masks')

# Move images and masks to validation folder
move_files('comma10k/imgs2', 'dataset_root/val/images')
move_files('comma10k/masks2', 'dataset_root/val/masks')

# Move SLAM *.py files to the dataset_root folder
move_selected_files( 'SLAM/', 'dataset_root/', prefix=None, suffix='.py' )

# List directories to verify
print(os.listdir('.'))
print(os.listdir('dataset_root'))


['.config', 'comma10k', 'dataset_root', 'drive', 'SLAM', 'sample_data']
['config.py', 'val', 'train', 'dataloader.py', 'train.py', 'model.py', 'utils.py', 'validate.py']


In [3]:
# Update filenames
import os

def add_suffix_to_mask_filenames(image_dir, mask_dir, suffix):
    image_files = os.listdir(image_dir)
    mask_files = os.listdir(mask_dir)

    for image_file in image_files:
        base, extension = os.path.splitext(image_file)
        expected_mask_file = f"{base}{suffix}{extension}"
        actual_mask_file = f"{base}{extension}"

        if actual_mask_file in mask_files:
            os.rename(os.path.join(mask_dir, actual_mask_file), os.path.join(mask_dir, expected_mask_file))
            print(f"Renamed {actual_mask_file} to {expected_mask_file}")


add_suffix_to_mask_filenames('/content/dataset_root/train/images', '/content/dataset_root/train/masks', '_mask')

Streaming output truncated to the last 5000 lines.
Renamed 5113_b5e785c1fc446ed0_2018-05-17--17-11-38_18_350.png to 5113_b5e785c1fc446ed0_2018-05-17--17-11-38_18_350_mask.png
Renamed 7651_23fca19e46950b0a_2018-08-03--17-43-32_59_175.png to 7651_23fca19e46950b0a_2018-08-03--17-43-32_59_175_mask.png
Renamed 5161_7201d920c64cc9fe_2019-04-02--12-29-52_3_916.png to 5161_7201d920c64cc9fe_2019-04-02--12-29-52_3_916_mask.png
Renamed 5826_886f3f426ddfd8c6_2018-11-04--15-06-31_32_10.png to 5826_886f3f426ddfd8c6_2018-11-04--15-06-31_32_10_mask.png
Renamed 0469_437cf078b5fcc611_2018-09-06--04-54-22_36_267.png to 0469_437cf078b5fcc611_2018-09-06--04-54-22_36_267_mask.png
Renamed 4583_f550e313a9262051_2018-07-30--19-06-48_5_112.png to 4583_f550e313a9262051_2018-07-30--19-06-48_5_112_mask.png
Renamed 1608_e1215f586d353b92_2018-10-06--16-04-11_23_594.png to 1608_e1215f586d353b92_2018-10-06--16-04-11_23_594_mask.png
Renamed 8969_e924475a6d31b24d_2019-01-18--01-20-53_17_665.png to 8969_e924475a6d31b24d_

In [4]:
# Verify all same number of files are in the folders

!ls -l dataset_root/train/images | grep ^- | wc -l
!ls -l dataset_root/train/masks | grep ^- | wc -l

!ls -l dataset_root/val/images | grep ^- | wc -l
!ls -l dataset_root/val/masks | grep ^- | wc -l

9888
9888
2000
2000


# Model Section

## Segmentation Model: U-net Segmentaiton

In [ ]:
!python dataset_root/train.py

Model Summary:
UNet(
  (enc_conv0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (enc_conv1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (enc_conv2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (enc_conv3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (bottleneck): Seque